importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
#Reading matched AVL to Timetable data 
df = pd.read_csv('./arrivalTimes_04_09.csv')
df.head()

,LineRef,BlockRef,Day,VehicleJourneyRef,DestinationAimedArrivalTime,JourneyCode,Total Distance,Run Time,FromStopID,FromStopLocation,ToStopID,ToStopLocation,AimedDepartureTime,AimedArrivalTime,ActualDepartureTime,ActualArrivalTime,RouteID
0,NT1,1001,Friday,NT1-In-1001-NT260007454-2024-08-23T06:36:00-20...,2024-08-23T06:37:00+00:00,1021.0,195.868599,PT0S,260007454,"(np.float64(-1.2052472), np.float64(52.7720907))",260007423,"(-1.202979, 52.7731152)",06:36:00,06:36:00,2024-08-23T05:30:12+00:00,2024-08-23T05:30:12+00:00,1:1_1_2147483284
1,NT1,1001,Friday,NT1-In-1001-NT260007454-2024-08-23T06:36:00-20...,2024-08-23T06:37:00+00:00,1021.0,346.400087,PT1M,260007423,"(-1.202979, 52.7731152)",260007343,"(-1.1997359, 52.7754058)",06:36:00,06:37:00,2024-08-23T05:30:12+00:00,2024-08-23T05:30:12+00:00,1:1_2_2147483283
2,NT1,1001,Friday,NT1-In-1001-NT260007454-2024-08-23T08:56:00-20...,2024-08-23T08:57:00+00:00,1053.0,195.868599,PT0S,260007454,"(np.float64(-1.2052472), np.float64(52.7720907))",260007423,"(-1.202979, 52.7731152)",08:56:00,08:56:00,2024-08-23T07:49:10+00:00,2024-08-23T07:58:33+00:00,1:1_1_2147483284
3,NT1,1001,Friday,NT1-In-1001-NT260007454-2024-08-23T08:56:00-20...,2024-08-23T08:57:00+00:00,1053.0,346.400087,PT1M,260007423,"(-1.202979, 52.7731152)",260007343,"(-1.1997359, 52.7754058)",08:56:00,08:57:00,2024-08-23T07:58:33+00:00,2024-08-23T08:00:23+00:00,1:1_2_2147483283
4,NT1,1001,Friday,NT1-In-1001-NT260007454-2024-08-23T11:14:00-20...,2024-08-23T11:15:00+00:00,1081.0,195.868599,PT0S,260007454,"(np.float64(-1.2052472), np.float64(52.7720907))",260007423,"(-1.202979, 52.7731152)",11:14:00,11:14:00,2024-08-23T10:10:10+00:00,2024-08-23T10:15:31+00:00,1:1_1_2147483284


In [3]:
df.shape

(9080, 17)

Preprocessing time-related attributes 

In [4]:

# Convert 'ActualDepartureTime' and 'ActualArrivalTime' to datetime european time 
df['ActualDepartureTime'] = pd.to_datetime(df['ActualDepartureTime'])
df['ActualArrivalTime'] = pd.to_datetime(df['ActualArrivalTime'])

# Convert from UTC to London time
df['ActualDepartureTime'] = df['ActualDepartureTime'].dt.tz_convert('Europe/London')
df['ActualArrivalTime'] = df['ActualArrivalTime'].dt.tz_convert('Europe/London')

# Display the DataFrame with the new London times
print(df[['ActualDepartureTime','ActualArrivalTime']])

           ActualDepartureTime         ActualArrivalTime
0    2024-08-23 06:30:12+01:00 2024-08-23 06:30:12+01:00
1    2024-08-23 06:30:12+01:00 2024-08-23 06:30:12+01:00
2    2024-08-23 08:49:10+01:00 2024-08-23 08:58:33+01:00
3    2024-08-23 08:58:33+01:00 2024-08-23 09:00:23+01:00
4    2024-08-23 11:10:10+01:00 2024-08-23 11:15:31+01:00
...                        ...                       ...
9075 2024-08-28 16:42:23+01:00 2024-08-28 16:43:23+01:00
9076 2024-08-28 16:43:23+01:00 2024-08-28 16:45:23+01:00
9077 2024-08-28 16:45:23+01:00 2024-08-28 16:45:53+01:00
9078 2024-08-28 16:45:53+01:00 2024-08-28 16:46:23+01:00
9079 2024-08-28 16:46:23+01:00 2024-08-28 16:48:23+01:00

[9080 rows x 2 columns]


<!-- Grouping by each vehicleJourneyRef, in a unique JourneyCode-->

In [5]:
# create date time columns for actual arrival and departure times
df['ActualDepartureDateTime'] = pd.to_datetime(df['ActualDepartureTime'])
df['ActualArrivalDateTime'] = pd.to_datetime(df['ActualArrivalTime'])
df.head()

,LineRef,BlockRef,Day,VehicleJourneyRef,DestinationAimedArrivalTime,JourneyCode,Total Distance,Run Time,FromStopID,FromStopLocation,ToStopID,ToStopLocation,AimedDepartureTime,AimedArrivalTime,ActualDepartureTime,ActualArrivalTime,RouteID,ActualDepartureDateTime,ActualArrivalDateTime
0,NT1,1001,Friday,NT1-In-1001-NT260007454-2024-08-23T06:36:00-20...,2024-08-23T06:37:00+00:00,1021.0,195.868599,PT0S,260007454,"(np.float64(-1.2052472), np.float64(52.7720907))",260007423,"(-1.202979, 52.7731152)",06:36:00,06:36:00,2024-08-23 06:30:12+01:00,2024-08-23 06:30:12+01:00,1:1_1_2147483284,2024-08-23 06:30:12+01:00,2024-08-23 06:30:12+01:00
1,NT1,1001,Friday,NT1-In-1001-NT260007454-2024-08-23T06:36:00-20...,2024-08-23T06:37:00+00:00,1021.0,346.400087,PT1M,260007423,"(-1.202979, 52.7731152)",260007343,"(-1.1997359, 52.7754058)",06:36:00,06:37:00,2024-08-23 06:30:12+01:00,2024-08-23 06:30:12+01:00,1:1_2_2147483283,2024-08-23 06:30:12+01:00,2024-08-23 06:30:12+01:00
2,NT1,1001,Friday,NT1-In-1001-NT260007454-2024-08-23T08:56:00-20...,2024-08-23T08:57:00+00:00,1053.0,195.868599,PT0S,260007454,"(np.float64(-1.2052472), np.float64(52.7720907))",260007423,"(-1.202979, 52.7731152)",08:56:00,08:56:00,2024-08-23 08:49:10+01:00,2024-08-23 08:58:33+01:00,1:1_1_2147483284,2024-08-23 08:49:10+01:00,2024-08-23 08:58:33+01:00
3,NT1,1001,Friday,NT1-In-1001-NT260007454-2024-08-23T08:56:00-20...,2024-08-23T08:57:00+00:00,1053.0,346.400087,PT1M,260007423,"(-1.202979, 52.7731152)",260007343,"(-1.1997359, 52.7754058)",08:56:00,08:57:00,2024-08-23 08:58:33+01:00,2024-08-23 09:00:23+01:00,1:1_2_2147483283,2024-08-23 08:58:33+01:00,2024-08-23 09:00:23+01:00
4,NT1,1001,Friday,NT1-In-1001-NT260007454-2024-08-23T11:14:00-20...,2024-08-23T11:15:00+00:00,1081.0,195.868599,PT0S,260007454,"(np.float64(-1.2052472), np.float64(52.7720907))",260007423,"(-1.202979, 52.7731152)",11:14:00,11:14:00,2024-08-23 11:10:10+01:00,2024-08-23 11:15:31+01:00,1:1_1_2147483284,2024-08-23 11:10:10+01:00,2024-08-23 11:15:31+01:00


Grouping by each vehicleJourneyRef, in a unique JourneyCode

Handling invalid actual arrival and  actual departure times data. For example, time after 23:30:00, Operation stops for NT1 buses, therefore records after that time result in same actual arrival and  actual departure times, thereby giving invalid data.

In [7]:
print(type(df['DestinationAimedArrivalTime'][0]))
print(type(df['ActualDepartureTime'][0]))
print(type(df['ActualArrivalTime'][0]))


<class 'str'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [8]:
# Convert 'DestinationAimedArrivalTime', 'ActualDepartureTime', and 'ActualArrivalTime' to datetime 
df['DestinationAimedArrivalTime'] =pd.to_datetime(df['DestinationAimedArrivalTime']).dt.time
df['ActualDepartureTime'] =pd.to_datetime(df['ActualDepartureTime']).dt.time
df['ActualArrivalTime'] =pd.to_datetime(df['ActualArrivalTime']).dt.time

print(type(df['DestinationAimedArrivalTime'][0]))
print(type(df['ActualDepartureTime'][0]))
print(type(df['ActualArrivalTime'][0]))


<class 'datetime.time'>
<class 'datetime.time'>
<class 'datetime.time'>


In [9]:
df.shape

(9080, 19)

Sorting data that is inconsistent, journey start time is recorded as journey end time.

In [11]:
# Identify rows where ActualDepartureTime is greater than ActualArrivalTime
affected_rows = df[df['ActualDepartureTime'] > df['ActualArrivalTime']]

# Display the affected rows
affected_rows


,LineRef,BlockRef,Day,VehicleJourneyRef,DestinationAimedArrivalTime,JourneyCode,Total Distance,Run Time,FromStopID,FromStopLocation,ToStopID,ToStopLocation,AimedDepartureTime,AimedArrivalTime,ActualDepartureTime,ActualArrivalTime,RouteID,ActualDepartureDateTime,ActualArrivalDateTime
1125,NT1,5001,Friday,NT1-Out-5001-NT3390B2-2024-08-23T09:00:00-2024...,09:06:00,1038.0,399.679171,PT1M,3390B2,"(np.float64(-1.1513667), np.float64(52.9533205))",3390M2,"(-1.1529839, 52.9522257)",09:00:00,09:01:00,09:00:50,08:57:29,1:27_1_2147483234,2024-08-23 09:00:50+01:00,2024-08-23 08:57:29+01:00
3423,NT1,2001,Thursday,NT1-Out-2001-NT3390B2-2024-08-23T00:02:00-2024...,23:30:00,1198.0,536.880316,PT1M,3390M2,"(-1.1529839, 52.9522257)",3390C2,"(-1.1493806, 52.94862)",00:02:00,00:03:00,23:56:01,00:04:54,1:25_2_12589,2024-08-22 23:56:01+01:00,2024-08-23 00:04:54+01:00
3454,NT1,2001,Thursday,NT1-Out-2001-NT3390B2-2024-08-23T00:02:00-2024...,23:30:00,1030.0,536.880316,PT1M,3390M2,"(-1.1529839, 52.9522257)",3390C2,"(-1.1493806, 52.94862)",08:31:00,08:32:00,23:56:01,00:04:54,1:26_2_12589,2024-08-22 23:56:01+01:00,2024-08-23 00:04:54+01:00
3498,NT1,2001,Thursday,NT1-Out-2001-NT3390B2-2024-08-23T00:02:00-2024...,23:30:00,1006.0,536.880316,PT1M,3390M2,"(-1.1529839, 52.9522257)",3390C2,"(-1.1493806, 52.94862)",06:16:00,06:17:00,23:56:01,00:04:54,1:27_2_12589,2024-08-22 23:56:01+01:00,2024-08-23 00:04:54+01:00
3556,NT1,2001,Thursday,NT1-Out-2001-NT3390B2-2024-08-23T00:02:00-2024...,23:30:00,1142.0,536.880316,PT1M,3390M2,"(-1.1529839, 52.9522257)",3390C2,"(-1.1493806, 52.94862)",17:41:00,17:42:00,23:56:01,00:04:54,1:27_2_12589,2024-08-22 23:56:01+01:00,2024-08-23 00:04:54+01:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7729,NT1,2001,Wednesday,NT1-Out-2001-NT3390B2-2024-08-22T00:02:00-2024...,23:30:00,1153.0,234.998596,PT1M,3390ME07,"(-1.1401671, 52.9409167)",3390ME08,"(-1.1403793, 52.9430261)",17:19:00,17:20:00,00:07:41,00:07:07,1:7_50_2147483240,2024-08-22 00:07:41+01:00,2024-08-22 00:07:07+01:00
7730,NT1,2001,Wednesday,NT1-Out-2001-NT3390B2-2024-08-22T00:02:00-2024...,23:30:00,1153.0,383.340659,PT1M,3390ME08,"(-1.1403793, 52.9430261)",3390ME09,"(-1.1442166, 52.9446762)",17:20:00,17:21:00,00:07:07,00:06:48,1:7_51_2147483239,2024-08-22 00:07:07+01:00,2024-08-22 00:06:48+01:00
7731,NT1,2001,Wednesday,NT1-Out-2001-NT3390B2-2024-08-22T00:02:00-2024...,23:30:00,1153.0,493.541864,PT2M,3390ME09,"(-1.1442166, 52.9446762)",3390S6,"(-1.1476178, 52.9479606)",17:21:00,17:23:00,00:06:48,00:04:42,1:7_52_2147483238,2024-08-22 00:06:48+01:00,2024-08-22 00:04:42+01:00
7732,NT1,2001,Wednesday,NT1-Out-2001-NT3390B2-2024-08-22T00:02:00-2024...,23:30:00,1153.0,433.442718,PT1M,3390S6,"(-1.1476178, 52.9479606)",3390M5,"(-1.1513415, 52.9501331)",17:23:00,17:24:00,00:04:42,00:03:53,1:7_53_4248,2024-08-22 00:04:42+01:00,2024-08-22 00:03:53+01:00


In [12]:
#Data preprocessing continued
original_df = df.copy()

# Function to swap times if ActualDepartureTime is greater than ActualArrivalTime
def swap_times(group):
    group = group.copy()  # Ensure we're working with a copy of the group
    for index, row in group.iterrows():
        if row['ActualDepartureTime'] > row['ActualArrivalTime']:
            group.at[index, 'ActualDepartureTime'], group.at[index, 'ActualArrivalTime'] = (
                row['ActualArrivalTime'],
                row['ActualDepartureTime'],
            )
    return group

# Apply the function to each group
df = df.groupby(['VehicleJourneyRef', 'JourneyCode']).apply(swap_times).reset_index(drop=True)

# Ensuring that the journey starts with the least ActualDepartureTime and ends with the greatest in each group
df = df.sort_values(by=['VehicleJourneyRef', 'JourneyCode', 'ActualDepartureTime'])


# Reset index in the original DataFrame to ensure they match
original_df.reset_index(drop=True, inplace=True)


/var/folders/gs/fx6m51n14jl6gggxb3q4vbpc0000gn/T/ipykernel_3005/3742787522.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['VehicleJourneyRef', 'JourneyCode']).apply(swap_times).reset_index(drop=True)


In [13]:
# Identify rows where ActualDepartureTime is greater than ActualArrivalTime
affected_rows = df[df['ActualDepartureTime'] > df['ActualArrivalTime']]

# Display the affected rows
affected_rows

,LineRef,BlockRef,Day,VehicleJourneyRef,DestinationAimedArrivalTime,JourneyCode,Total Distance,Run Time,FromStopID,FromStopLocation,ToStopID,ToStopLocation,AimedDepartureTime,AimedArrivalTime,ActualDepartureTime,ActualArrivalTime,RouteID,ActualDepartureDateTime,ActualArrivalDateTime


In [14]:
#repeating the same process for AimedDeparture and Aimedarrival times
# Function to swap times if ActualDepartureTime is greater than ActualArrivalTime
def swap_times(group):
    group = group.copy()  # Ensure we're working with a copy of the group
    for index, row in group.iterrows():
        if row['AimedDepartureTime'] > row['AimedArrivalTime']:
            group.at[index, 'AimedDepartureTime'], group.at[index, 'ActualArrivalTime'] = (
                row['AimedArrivalTime'],
                row['AimedDepartureTime'],
            )
    return group

# Apply the function to each group
df = df.groupby(['VehicleJourneyRef', 'JourneyCode']).apply(swap_times).reset_index(drop=True)


/var/folders/gs/fx6m51n14jl6gggxb3q4vbpc0000gn/T/ipykernel_3005/593296512.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['VehicleJourneyRef', 'JourneyCode']).apply(swap_times).reset_index(drop=True)


In [15]:
# Identify rows where ActualDepartureTime is greater than ActualArrivalTime
affected_rows = df[df['AimedDepartureTime'] > df['AimedArrivalTime']]

# Display the affected rows
affected_rows

,LineRef,BlockRef,Day,VehicleJourneyRef,DestinationAimedArrivalTime,JourneyCode,Total Distance,Run Time,FromStopID,FromStopLocation,ToStopID,ToStopLocation,AimedDepartureTime,AimedArrivalTime,ActualDepartureTime,ActualArrivalTime,RouteID,ActualDepartureDateTime,ActualArrivalDateTime


In [16]:
df.shape

(9080, 19)

In [17]:
# identify rows where the time difference exceeds 43 minutes
from datetime import datetime

#converting Aimed departure and arrival times to the right format
df['AimedDepartureTime'] =pd.to_datetime(df['AimedDepartureTime']).dt.time
df['AimedArrivalTime'] =pd.to_datetime(df['AimedArrivalTime']).dt.time


# Function to identify rows where the time difference exceeds 43 minutes
def identify_invalid_times(df):
    affected_rows = []
    
    for index, row in df.iterrows():
        # Convert ActualDepartureTime and ActualArrivalTime to datetime objects with a common date
        aimed_departure = datetime.combine(datetime.min, row['AimedDepartureTime'])
        aimed_arrival = datetime.combine(datetime.min, row['AimedArrivalTime'])
        
        # Calculate the time difference between ActualArrivalTime and ActualDepartureTime
        time_difference = abs(aimed_arrival - aimed_departure)
        
        # Check if the time difference is greater than 43 minutes (2580 seconds)
        if time_difference.total_seconds() > 2580:  # 43 minutes * 60 seconds/minute
            affected_rows.append(index)  # Marking row as affected
    
    return df.loc[affected_rows] 

# Get the indices of the affected rows
affected_rows = identify_invalid_times(df)



# Show the affected columns
affected_rows


/var/folders/gs/fx6m51n14jl6gggxb3q4vbpc0000gn/T/ipykernel_3005/2442622555.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['AimedDepartureTime'] =pd.to_datetime(df['AimedDepartureTime']).dt.time


/var/folders/gs/fx6m51n14jl6gggxb3q4vbpc0000gn/T/ipykernel_3005/2442622555.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['AimedArrivalTime'] =pd.to_datetime(df['AimedArrivalTime']).dt.time


,LineRef,BlockRef,Day,VehicleJourneyRef,DestinationAimedArrivalTime,JourneyCode,Total Distance,Run Time,FromStopID,FromStopLocation,ToStopID,ToStopLocation,AimedDepartureTime,AimedArrivalTime,ActualDepartureTime,ActualArrivalTime,RouteID,ActualDepartureDateTime,ActualArrivalDateTime
953,NT1,2001,Tuesday,NT1-Out-2001-NT3390B2-2024-08-21T00:02:00-2024...,23:30:00,1030.0,399.679171,PT1M,3300RU0381,"(-1.2055591, 52.8645683)",3390M2,"(-1.1529839, 52.9522257)",00:30:00,08:31:00,00:30:46,00:30:46,1:26_1_2147483234,2024-08-21 00:30:46+01:00,2024-08-21 00:30:46+01:00
1149,NT1,2001,Tuesday,NT1-Out-2001-NT3390B2-2024-08-21T00:02:00-2024...,23:30:00,1082.0,399.679171,PT1M,3390M2,"(-1.1529839, 52.9522257)",3390M2,"(-1.1529839, 52.9522257)",10:21:00,12:41:00,00:30:46,00:30:46,1:27_1_2147483234,2024-08-21 00:30:46+01:00,2024-08-21 00:30:46+01:00
1150,NT1,2001,Tuesday,NT1-Out-2001-NT3390B2-2024-08-21T00:02:00-2024...,23:30:00,1082.0,536.880316,PT1M,3390C2,"(-1.1493806, 52.94862)",3390C2,"(-1.1493806, 52.94862)",10:22:00,12:42:00,00:30:46,00:30:46,1:27_2_12589,2024-08-21 00:30:46+01:00,2024-08-21 00:30:46+01:00
1151,NT1,2001,Tuesday,NT1-Out-2001-NT3390B2-2024-08-21T00:02:00-2024...,23:30:00,1082.0,195.439142,PT2M,3390S2,"(-1.147587, 52.9478946)",3390S2,"(-1.147587, 52.9478946)",10:24:00,12:44:00,00:30:46,00:30:46,1:27_3_2147480598,2024-08-21 00:30:46+01:00,2024-08-21 00:30:46+01:00
1152,NT1,2001,Tuesday,NT1-Out-2001-NT3390B2-2024-08-21T00:02:00-2024...,23:30:00,1082.0,216.925873,PT0S,3390ME01,"(-1.14670255151079, 52.94601864708672)",3390ME01,"(-1.14670255151079, 52.94601864708672)",10:24:00,12:44:00,00:30:46,00:30:46,1:27_4_2147483231,2024-08-21 00:30:46+01:00,2024-08-21 00:30:46+01:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8377,NT1,4001,Saturday,NT1-Out-4001-NT3390B2-2024-09-01T00:02:00-2024...,23:30:00,2181.0,394.888771,PT1M,3300RU0366,"(-1.2036151, 52.8701081)",3300RU0366,"(-1.2036151, 52.8701081)",21:26:00,23:26:00,00:27:53,00:27:53,1:1_27_2147483260,2024-09-01 00:27:53+01:00,2024-09-01 00:27:53+01:00
8378,NT1,4001,Saturday,NT1-Out-4001-NT3390B2-2024-09-01T00:02:00-2024...,23:30:00,2181.0,534.855378,PT1M,3300RU0364,"(-1.2016642, 52.8744198)",3300RU0364,"(-1.2016642, 52.8744198)",21:27:00,23:27:00,00:27:53,00:27:53,1:1_28_2147483259,2024-09-01 00:27:53+01:00,2024-09-01 00:27:53+01:00
8379,NT1,4001,Saturday,NT1-Out-4001-NT3390B2-2024-09-01T00:02:00-2024...,23:30:00,2181.0,1362.452109,PT1M,3300RU0961,"(-1.2001045, 52.8863978)",3300RU0961,"(-1.2001045, 52.8863978)",21:28:00,23:28:00,00:27:53,00:27:53,1:1_29_4593,2024-09-01 00:27:53+01:00,2024-09-01 00:27:53+01:00
8380,NT1,4001,Saturday,NT1-Out-4001-NT3390B2-2024-09-01T00:02:00-2024...,23:30:00,2181.0,874.119513,PT1M,3300RU0970,"(-1.194495, 52.8929294)",3300RU0970,"(-1.194495, 52.8929294)",21:29:00,23:29:00,00:27:53,00:27:53,1:1_30_9186,2024-09-01 00:27:53+01:00,2024-09-01 00:27:53+01:00


In [18]:
from datetime import datetime, timedelta
import isodate  # To handle ISO 8601 durations

# Function to adjust AimedDepartureTime and calculate new AimedArrivalTime
def adjust_times(group):
    group = group.copy()  # Copy the group to avoid modifying the original data
    
    # Iterate over the rows and apply the adjustment
    for index, row in group.iterrows():
        # Calculate the time difference between AimedDepartureTime and AimedArrivalTime
        aimed_departure = datetime.combine(datetime.min, row['AimedDepartureTime'])
        aimed_arrival = datetime.combine(datetime.min, row['AimedArrivalTime'])
        time_difference = abs(aimed_arrival - aimed_departure)
        
        # Apply the function only if the time difference is greater than 43 minutes (2580 seconds)
        if time_difference.total_seconds() > 2580:  # 43 minutes = 2580 seconds
            # Replace AimedDepartureTime with ActualDepartureTime
            actual_departure = datetime.combine(datetime.min, row['ActualDepartureTime'])
            
            # Parse Run Time (as it's in ISO 8601 duration format 'PT1M')
            run_time = isodate.parse_duration(row['Run Time'])
            
            # Set the new AimedDepartureTime to the ActualDepartureTime
            group.at[index, 'AimedDepartureTime'] = actual_departure.time()
            
            # Calculate the new AimedArrivalTime by adding the Run Time
            new_aimed_arrival = actual_departure + run_time
            group.at[index, 'AimedArrivalTime'] = new_aimed_arrival.time()
    
    return group

# Apply the function to each group based on VehicleJourneyRef and JourneyCode
df_modified = df.groupby(['VehicleJourneyRef', 'JourneyCode']).apply(adjust_times).reset_index(drop=True)

# Check the result
df_modified.head()


/var/folders/gs/fx6m51n14jl6gggxb3q4vbpc0000gn/T/ipykernel_3005/2763500575.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_modified = df.groupby(['VehicleJourneyRef', 'JourneyCode']).apply(adjust_times).reset_index(drop=True)


,LineRef,BlockRef,Day,VehicleJourneyRef,DestinationAimedArrivalTime,JourneyCode,Total Distance,Run Time,FromStopID,FromStopLocation,ToStopID,ToStopLocation,AimedDepartureTime,AimedArrivalTime,ActualDepartureTime,ActualArrivalTime,RouteID,ActualDepartureDateTime,ActualArrivalDateTime
0,NT1,1001,Wednesday,NT1-In-1001-NT260007454-2024-08-21T06:36:00-20...,06:37:00,1021.0,195.868599,PT0S,260007454,"(np.float64(-1.2052472), np.float64(52.7720907))",260007423,"(-1.202979, 52.7731152)",06:36:00,06:36:00,06:26:33,06:35:51,1:1_1_2147483284,2024-08-21 06:26:33+01:00,2024-08-21 06:35:51+01:00
1,NT1,1001,Wednesday,NT1-In-1001-NT260007454-2024-08-21T06:36:00-20...,06:37:00,1021.0,346.400087,PT1M,260007423,"(-1.202979, 52.7731152)",260007343,"(-1.1997359, 52.7754058)",06:36:00,06:37:00,06:35:51,06:37:42,1:1_2_2147483283,2024-08-21 06:35:51+01:00,2024-08-21 06:37:42+01:00
2,NT1,1001,Wednesday,NT1-In-1001-NT260007454-2024-08-21T08:56:00-20...,08:57:00,1053.0,195.868599,PT0S,260007454,"(np.float64(-1.2052472), np.float64(52.7720907))",260007423,"(-1.202979, 52.7731152)",08:56:00,08:56:00,08:51:41,08:57:05,1:1_1_2147483284,2024-08-21 08:51:41+01:00,2024-08-21 08:57:05+01:00
3,NT1,1001,Wednesday,NT1-In-1001-NT260007454-2024-08-21T08:56:00-20...,08:57:00,1053.0,346.400087,PT1M,260007423,"(-1.202979, 52.7731152)",260007343,"(-1.1997359, 52.7754058)",08:56:00,08:57:00,08:57:05,08:59:23,1:1_2_2147483283,2024-08-21 08:57:05+01:00,2024-08-21 08:59:23+01:00
4,NT1,1001,Wednesday,NT1-In-1001-NT260007454-2024-08-21T11:14:00-20...,11:15:00,1081.0,195.868599,PT0S,260007454,"(np.float64(-1.2052472), np.float64(52.7720907))",260007423,"(-1.202979, 52.7731152)",11:14:00,11:14:00,11:06:27,11:06:27,1:1_1_2147483284,2024-08-21 11:06:27+01:00,2024-08-21 11:06:27+01:00


In [20]:
df_modified.shape

(9080, 19)

In [22]:
#assigning modified dataframe as the new dataframe 
df = df_modified
# Function to check if all departure and arrival times are the same within a group
def check_and_drop_rows(group):
    if group['ActualDepartureDateTime'].equals(group['ActualArrivalDateTime']):
        return pd.DataFrame()  # Drop all rows in this case
    else:
        return group

# Apply the function to each group
df_cleaned = df.groupby(['VehicleJourneyRef', 'JourneyCode']).apply(check_and_drop_rows).reset_index(drop=True)

# Display the cleaned DataFrame
print("Cleaned DataFrame:")
display(df_cleaned)

Cleaned DataFrame:


/var/folders/gs/fx6m51n14jl6gggxb3q4vbpc0000gn/T/ipykernel_3005/1131592777.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_cleaned = df.groupby(['VehicleJourneyRef', 'JourneyCode']).apply(check_and_drop_rows).reset_index(drop=True)


,LineRef,BlockRef,Day,VehicleJourneyRef,DestinationAimedArrivalTime,JourneyCode,Total Distance,Run Time,FromStopID,FromStopLocation,ToStopID,ToStopLocation,AimedDepartureTime,AimedArrivalTime,ActualDepartureTime,ActualArrivalTime,RouteID,ActualDepartureDateTime,ActualArrivalDateTime
0,NT1,1001.0,Wednesday,NT1-In-1001-NT260007454-2024-08-21T06:36:00-20...,06:37:00,1021.0,195.868599,PT0S,260007454,"(np.float64(-1.2052472), np.float64(52.7720907))",260007423,"(-1.202979, 52.7731152)",06:36:00,06:36:00,06:26:33,06:35:51,1:1_1_2147483284,2024-08-21 06:26:33+01:00,2024-08-21 06:35:51+01:00
1,NT1,1001.0,Wednesday,NT1-In-1001-NT260007454-2024-08-21T06:36:00-20...,06:37:00,1021.0,346.400087,PT1M,260007423,"(-1.202979, 52.7731152)",260007343,"(-1.1997359, 52.7754058)",06:36:00,06:37:00,06:35:51,06:37:42,1:1_2_2147483283,2024-08-21 06:35:51+01:00,2024-08-21 06:37:42+01:00
2,NT1,1001.0,Wednesday,NT1-In-1001-NT260007454-2024-08-21T08:56:00-20...,08:57:00,1053.0,195.868599,PT0S,260007454,"(np.float64(-1.2052472), np.float64(52.7720907))",260007423,"(-1.202979, 52.7731152)",08:56:00,08:56:00,08:51:41,08:57:05,1:1_1_2147483284,2024-08-21 08:51:41+01:00,2024-08-21 08:57:05+01:00
3,NT1,1001.0,Wednesday,NT1-In-1001-NT260007454-2024-08-21T08:56:00-20...,08:57:00,1053.0,346.400087,PT1M,260007423,"(-1.202979, 52.7731152)",260007343,"(-1.1997359, 52.7754058)",08:56:00,08:57:00,08:57:05,08:59:23,1:1_2_2147483283,2024-08-21 08:57:05+01:00,2024-08-21 08:59:23+01:00
4,NT1,1001.0,Wednesday,NT1-In-1001-NT260007454-2024-08-21T11:14:00-20...,11:15:00,1081.0,195.868599,PT0S,260007454,"(np.float64(-1.2052472), np.float64(52.7720907))",260007423,"(-1.202979, 52.7731152)",11:14:00,11:14:00,11:06:27,11:06:27,1:1_1_2147483284,2024-08-21 11:06:27+01:00,2024-08-21 11:06:27+01:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3462,NT1,7001.0,Monday,NT1-Out-7001-NT3390B2-2024-09-02T16:40:00-2024...,16:51:00,1130.0,195.439142,PT3M,3390C2,"(-1.1493806, 52.94862)",3390S2,"(-1.147587, 52.9478946)",16:42:00,16:45:00,16:42:03,16:42:03,1:27_3_2147480598,2024-09-02 16:42:03+01:00,2024-09-02 16:42:03+01:00
3463,NT1,7001.0,Monday,NT1-Out-7001-NT3390B2-2024-09-02T16:40:00-2024...,16:51:00,1130.0,216.925873,PT0S,3390S2,"(-1.147587, 52.9478946)",3390ME01,"(-1.14670255151079, 52.94601864708672)",16:45:00,16:45:00,16:42:03,16:42:03,1:27_4_2147483231,2024-09-02 16:42:03+01:00,2024-09-02 16:42:03+01:00
3464,NT1,7001.0,Monday,NT1-Out-7001-NT3390B2-2024-09-02T16:40:00-2024...,16:51:00,1130.0,340.096907,PT1M,3390ME01,"(-1.14670255151079, 52.94601864708672)",3390ME02,"(-1.1431284, 52.944636)",16:45:00,16:46:00,16:42:03,16:42:03,1:27_5_2147483230,2024-09-02 16:42:03+01:00,2024-09-02 16:42:03+01:00
3465,NT1,7001.0,Monday,NT1-Out-7001-NT3390B2-2024-09-02T16:40:00-2024...,16:51:00,1130.0,241.104345,PT1M,3390ME02,"(-1.1431284, 52.944636)",3390ME03,"(-1.1404309, 52.9436469)",16:46:00,16:47:00,16:42:03,16:42:03,1:27_6_2147483229,2024-09-02 16:42:03+01:00,2024-09-02 16:42:03+01:00


In [26]:
#assigning cleaned dataframe as the new dataframe 
df= df_cleaned
df.shape
# Convert 'ActualDepartureTime' and 'ActualArrivalTime' to datetime objects
df['ActualDepartureTime'] = pd.to_datetime(df['ActualDepartureTime'], format='%H:%M:%S', errors='coerce')
df['ActualArrivalTime'] = pd.to_datetime(df['ActualArrivalTime'], format='%H:%M:%S', errors='coerce')

# Calculate the time difference in seconds and identify rows where this difference is greater than 2580 seconds (43 minutes)
time_difference = (df['ActualArrivalTime'] - df['ActualDepartureTime']).abs()
outliers = time_difference > timedelta(seconds=2580)

# Display the rows that are identified as outliers (to be dropped)
outlier_rows = df[outliers]
print("Outlier Rows:")
display(outlier_rows)

# Drop the outliers from the DataFrame
df_cleaned = df[~outliers]

# Display the cleaned DataFrame
print("Cleaned DataFrame:")
display(df_cleaned)


Outlier Rows:


,LineRef,BlockRef,Day,VehicleJourneyRef,DestinationAimedArrivalTime,JourneyCode,Total Distance,Run Time,FromStopID,FromStopLocation,ToStopID,ToStopLocation,AimedDepartureTime,AimedArrivalTime,ActualDepartureTime,ActualArrivalTime,RouteID,ActualDepartureDateTime,ActualArrivalDateTime
541,NT1,1001.0,Tuesday,NT1-Out-1001-NT3390B2-2024-08-27T10:00:00-2024...,10:06:00,1050.0,536.880316,PT1M,3390M2,"(-1.1529839, 52.9522257)",3390C2,"(-1.1493806, 52.94862)",10:01:00,10:02:00,1900-01-01 10:04:04,1900-01-01 11:05:20,1:27_2_12589,2024-08-27 11:05:20+01:00,2024-08-27 10:04:04+01:00
663,NT1,2001.0,Wednesday,NT1-Out-2001-NT3390B2-2024-08-22T00:02:00-2024...,23:30:00,1005.0,440.782773,PT2M,3390M5,"(-1.1513415, 52.9501331)",3390B2,"(-1.1513667, 52.9533205)",06:09:00,06:11:00,1900-01-01 00:03:53,1900-01-01 23:56:13,1:3_53_12571,2024-08-22 00:03:53+01:00,2024-08-21 23:56:13+01:00
685,NT1,2001.0,Wednesday,NT1-Out-2001-NT3390B2-2024-08-22T00:02:00-2024...,23:30:00,1005.0,332.687812,PT0S,3390B2,"(-1.1513667, 52.9533205)",3300RU0368,"(-1.2056744, 52.8670924)",05:40:00,05:40:00,1900-01-01 00:23:51,1900-01-01 10:15:00,1:3_26_2147483261,2024-08-21 23:56:13+01:00,2024-08-22 00:23:51+01:00
690,NT1,2001.0,Wednesday,NT1-Out-2001-NT3390B2-2024-08-22T00:02:00-2024...,23:30:00,1006.0,399.679171,PT1M,3300RU0415,"(-1.1871586, 52.8238621)",3390M2,"(-1.1529839, 52.9522257)",06:16:00,06:16:00,1900-01-01 00:03:24,1900-01-01 09:17:00,1:27_1_2147483234,2024-08-22 00:23:51+01:00,2024-08-22 00:03:24+01:00
793,NT1,2001.0,Wednesday,NT1-Out-2001-NT3390B2-2024-08-22T00:02:00-2024...,23:30:00,1033.0,440.782773,PT4M,3390M5,"(-1.1513415, 52.9501331)",3390B2,"(-1.1513667, 52.9533205)",08:23:00,08:27:00,1900-01-01 00:03:53,1900-01-01 23:56:13,1:1_53_12571,2024-08-22 00:03:53+01:00,2024-08-21 23:56:13+01:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587,NT1,2001.0,Thursday,NT1-Out-2001-NT3390B2-2024-08-30T00:02:00-2024...,23:30:00,1177.0,354.395284,PT1M,3300RU0345,"(-1.1866801, 52.8485518)",3300RU0344,"(-1.1893708, 52.851292)",19:21:00,19:21:00,1900-01-01 00:28:17,1900-01-01 21:20:00,1:1_22_2147483265,2024-08-30 00:28:17+01:00,2024-08-30 00:28:17+01:00
2588,NT1,2001.0,Thursday,NT1-Out-2001-NT3390B2-2024-08-30T00:02:00-2024...,23:30:00,1177.0,1245.500852,PT1M,3300RU0344,"(-1.1893708, 52.851292)",3300RU0342,"(-1.201853, 52.8594477)",19:22:00,19:22:00,1900-01-01 00:28:17,1900-01-01 21:21:00,1:1_23_2147483264,2024-08-30 00:28:17+01:00,2024-08-30 00:28:17+01:00
2589,NT1,2001.0,Thursday,NT1-Out-2001-NT3390B2-2024-08-30T00:02:00-2024...,23:30:00,1177.0,390.588173,PT1M,3300RU0342,"(-1.201853, 52.8594477)",3300RU0708,"(-1.2051818, 52.8622134)",19:23:00,19:23:00,1900-01-01 00:28:17,1900-01-01 21:22:00,1:1_24_26209,2024-08-30 00:28:17+01:00,2024-08-30 00:28:17+01:00
2590,NT1,2001.0,Thursday,NT1-Out-2001-NT3390B2-2024-08-30T00:02:00-2024...,23:30:00,1177.0,230.775816,PT2M,3300RU0708,"(-1.2051818, 52.8622134)",3300RU0370,"(-1.2056151, 52.8641973)",19:25:00,19:25:00,1900-01-01 00:28:17,1900-01-01 21:23:00,1:1_25_29957,2024-08-30 00:28:17+01:00,2024-08-30 00:28:17+01:00


Cleaned DataFrame:


,LineRef,BlockRef,Day,VehicleJourneyRef,DestinationAimedArrivalTime,JourneyCode,Total Distance,Run Time,FromStopID,FromStopLocation,ToStopID,ToStopLocation,AimedDepartureTime,AimedArrivalTime,ActualDepartureTime,ActualArrivalTime,RouteID,ActualDepartureDateTime,ActualArrivalDateTime
0,NT1,1001.0,Wednesday,NT1-In-1001-NT260007454-2024-08-21T06:36:00-20...,06:37:00,1021.0,195.868599,PT0S,260007454,"(np.float64(-1.2052472), np.float64(52.7720907))",260007423,"(-1.202979, 52.7731152)",06:36:00,06:36:00,1900-01-01 06:26:33,1900-01-01 06:35:51,1:1_1_2147483284,2024-08-21 06:26:33+01:00,2024-08-21 06:35:51+01:00
1,NT1,1001.0,Wednesday,NT1-In-1001-NT260007454-2024-08-21T06:36:00-20...,06:37:00,1021.0,346.400087,PT1M,260007423,"(-1.202979, 52.7731152)",260007343,"(-1.1997359, 52.7754058)",06:36:00,06:37:00,1900-01-01 06:35:51,1900-01-01 06:37:42,1:1_2_2147483283,2024-08-21 06:35:51+01:00,2024-08-21 06:37:42+01:00
2,NT1,1001.0,Wednesday,NT1-In-1001-NT260007454-2024-08-21T08:56:00-20...,08:57:00,1053.0,195.868599,PT0S,260007454,"(np.float64(-1.2052472), np.float64(52.7720907))",260007423,"(-1.202979, 52.7731152)",08:56:00,08:56:00,1900-01-01 08:51:41,1900-01-01 08:57:05,1:1_1_2147483284,2024-08-21 08:51:41+01:00,2024-08-21 08:57:05+01:00
3,NT1,1001.0,Wednesday,NT1-In-1001-NT260007454-2024-08-21T08:56:00-20...,08:57:00,1053.0,346.400087,PT1M,260007423,"(-1.202979, 52.7731152)",260007343,"(-1.1997359, 52.7754058)",08:56:00,08:57:00,1900-01-01 08:57:05,1900-01-01 08:59:23,1:1_2_2147483283,2024-08-21 08:57:05+01:00,2024-08-21 08:59:23+01:00
4,NT1,1001.0,Wednesday,NT1-In-1001-NT260007454-2024-08-21T11:14:00-20...,11:15:00,1081.0,195.868599,PT0S,260007454,"(np.float64(-1.2052472), np.float64(52.7720907))",260007423,"(-1.202979, 52.7731152)",11:14:00,11:14:00,1900-01-01 11:06:27,1900-01-01 11:06:27,1:1_1_2147483284,2024-08-21 11:06:27+01:00,2024-08-21 11:06:27+01:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3462,NT1,7001.0,Monday,NT1-Out-7001-NT3390B2-2024-09-02T16:40:00-2024...,16:51:00,1130.0,195.439142,PT3M,3390C2,"(-1.1493806, 52.94862)",3390S2,"(-1.147587, 52.9478946)",16:42:00,16:45:00,1900-01-01 16:42:03,1900-01-01 16:42:03,1:27_3_2147480598,2024-09-02 16:42:03+01:00,2024-09-02 16:42:03+01:00
3463,NT1,7001.0,Monday,NT1-Out-7001-NT3390B2-2024-09-02T16:40:00-2024...,16:51:00,1130.0,216.925873,PT0S,3390S2,"(-1.147587, 52.9478946)",3390ME01,"(-1.14670255151079, 52.94601864708672)",16:45:00,16:45:00,1900-01-01 16:42:03,1900-01-01 16:42:03,1:27_4_2147483231,2024-09-02 16:42:03+01:00,2024-09-02 16:42:03+01:00
3464,NT1,7001.0,Monday,NT1-Out-7001-NT3390B2-2024-09-02T16:40:00-2024...,16:51:00,1130.0,340.096907,PT1M,3390ME01,"(-1.14670255151079, 52.94601864708672)",3390ME02,"(-1.1431284, 52.944636)",16:45:00,16:46:00,1900-01-01 16:42:03,1900-01-01 16:42:03,1:27_5_2147483230,2024-09-02 16:42:03+01:00,2024-09-02 16:42:03+01:00
3465,NT1,7001.0,Monday,NT1-Out-7001-NT3390B2-2024-09-02T16:40:00-2024...,16:51:00,1130.0,241.104345,PT1M,3390ME02,"(-1.1431284, 52.944636)",3390ME03,"(-1.1404309, 52.9436469)",16:46:00,16:47:00,1900-01-01 16:42:03,1900-01-01 16:42:03,1:27_6_2147483229,2024-09-02 16:42:03+01:00,2024-09-02 16:42:03+01:00


In [29]:
# Convert the 'ActualDepartureDateTime' and 'ActualArrivalDateTime' to just the time component
df['ActualDepartureTime'] = df['ActualDepartureDateTime'].dt.time
df['ActualArrivalTime'] = df['ActualArrivalDateTime'].dt.time

# Display the updated DataFrame
df[['ActualDepartureTime', 'ActualArrivalTime']].head()


,ActualDepartureTime,ActualArrivalTime
0,06:26:33,06:35:51
1,06:35:51,06:37:42
2,08:51:41,08:57:05
3,08:57:05,08:59:23
4,11:06:27,11:06:27


In [30]:
df.head()

,LineRef,BlockRef,Day,VehicleJourneyRef,DestinationAimedArrivalTime,JourneyCode,Total Distance,Run Time,FromStopID,FromStopLocation,ToStopID,ToStopLocation,AimedDepartureTime,AimedArrivalTime,ActualDepartureTime,ActualArrivalTime,RouteID,ActualDepartureDateTime,ActualArrivalDateTime
0,NT1,1001.0,Wednesday,NT1-In-1001-NT260007454-2024-08-21T06:36:00-20...,06:37:00,1021.0,195.868599,PT0S,260007454,"(np.float64(-1.2052472), np.float64(52.7720907))",260007423,"(-1.202979, 52.7731152)",06:36:00,06:36:00,06:26:33,06:35:51,1:1_1_2147483284,2024-08-21 06:26:33+01:00,2024-08-21 06:35:51+01:00
1,NT1,1001.0,Wednesday,NT1-In-1001-NT260007454-2024-08-21T06:36:00-20...,06:37:00,1021.0,346.400087,PT1M,260007423,"(-1.202979, 52.7731152)",260007343,"(-1.1997359, 52.7754058)",06:36:00,06:37:00,06:35:51,06:37:42,1:1_2_2147483283,2024-08-21 06:35:51+01:00,2024-08-21 06:37:42+01:00
2,NT1,1001.0,Wednesday,NT1-In-1001-NT260007454-2024-08-21T08:56:00-20...,08:57:00,1053.0,195.868599,PT0S,260007454,"(np.float64(-1.2052472), np.float64(52.7720907))",260007423,"(-1.202979, 52.7731152)",08:56:00,08:56:00,08:51:41,08:57:05,1:1_1_2147483284,2024-08-21 08:51:41+01:00,2024-08-21 08:57:05+01:00
3,NT1,1001.0,Wednesday,NT1-In-1001-NT260007454-2024-08-21T08:56:00-20...,08:57:00,1053.0,346.400087,PT1M,260007423,"(-1.202979, 52.7731152)",260007343,"(-1.1997359, 52.7754058)",08:56:00,08:57:00,08:57:05,08:59:23,1:1_2_2147483283,2024-08-21 08:57:05+01:00,2024-08-21 08:59:23+01:00
4,NT1,1001.0,Wednesday,NT1-In-1001-NT260007454-2024-08-21T11:14:00-20...,11:15:00,1081.0,195.868599,PT0S,260007454,"(np.float64(-1.2052472), np.float64(52.7720907))",260007423,"(-1.202979, 52.7731152)",11:14:00,11:14:00,11:06:27,11:06:27,1:1_1_2147483284,2024-08-21 11:06:27+01:00,2024-08-21 11:06:27+01:00


In [31]:
df_cleaned.to_csv('./clean_data.csv', index=False)

In [32]:
df_cleaned.shape

(2941, 19)